In [1]:
## 12 다중머신과 장치를 위한 분산 텐서플로 ##
import tensorflow as tf

In [2]:
!nvidia-smi

Tue Nov 10 13:07:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.67       Driver Version: 451.67       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 106... WDDM  | 00000000:06:00.0  On |                  N/A |
|  0%   43C    P8    12W / 120W |    410MiB /  3072MiB |     12%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [3]:
## GPU 메모리 관리 ##
#1번 방법
config = tf.ConfigProto()
config.gpu_options.allow_growth = True #메모리 수요에 따라 할당

'''주의할 점은, 이 옵션은 메모리의 증식만 가능하다는 것. 연산이 끝나고 메모리가 필요없는 상황이라고 해서 할당된 메모리를 반납하지 않는다. 

Tensorflow 측에서는 그럴 경우 더 심한 메모리 파편화를 일으킬 수도 있다고 하니 판단은 독자의 몫.'''

'주의할 점은, 이 옵션은 메모리의 증식만 가능하다는 것. 연산이 끝나고 메모리가 필요없는 상황이라고 해서 할당된 메모리를 반납하지 않는다. \n\nTensorflow 측에서는 그럴 경우 더 심한 메모리 파편화를 일으킬 수도 있다고 하니 판단은 독자의 몫.'

In [ ]:
## GPU 메모리 관리 ##
#2번 방법
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3 #메모리 할당 비율 설정
#config.gpu_options.allow_growth = True #메모리 수요에 따라 할당, 두개 같이 사용 가능(추천하지는 않음)

In [ ]:
config.log_device_placement = True #배치자가 노드를 배치할 때마다 메세지 기록

In [4]:
tf.reset_default_graph()

with tf.device("/cpu:0"): #변수 a와 b를 cpu에 할당
    a = tf.Variable(3.0)
    b = tf.Variable(4.0)
    
c = a * b #c는 어떤 장치에도 할당되지 않았으므로 기본장치에 배치됨

with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(c))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1060 3GB, pci bus id: 0000:06:00.0, compute capability: 6.1

12.0


In [7]:
## 동적배치함수 ##
def variables_on_cpu(op): #연산이 할당될 장치이름 반환
    if op.type == "Variable":
        return "/cpu:0"
    else:
        return "/gpu:0"

tf.reset_default_graph()

with tf.device(variables_on_cpu):
    a = tf.Variable(3.0) #타입이 Variable이므로 cpu에 할당됨
    b = tf.Variable(4.0) #타입이 Variable이므로 cpu에 할당됨
    #b = tf.Variable(4) #gpu에는 int32형 변수 할당불가능
    c = a * b 
    
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(c))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1060 3GB, pci bus id: 0000:06:00.0, compute capability: 6.1

12.0


In [18]:
## 간접배치 ##
tf.reset_default_graph()

config.allow_soft_placement = True #처리할 커널이 없는 장치에 할당했을 때 cpu로 처리할수 있도록 함

In [19]:
tf.reset_default_graph()

with tf.device("/gpu:0"):
    b = tf.Variable(4) #cpu에 할당될 것임
    
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(b))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1060 3GB, pci bus id: 0000:06:00.0, compute capability: 6.1

4


In [22]:
## 제어의존성 ##
tf.reset_default_graph()

a = tf.Variable(1.0)
b = a + 2.0

with tf.control_dependencies([a, b]): #a,b가 평가된 두 x, y가 평가되게 함
    x = tf.constant(3.0)
    y = tf.constant(4.0)

z = x * y * b

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(z))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1060 3GB, pci bus id: 0000:06:00.0, compute capability: 6.1

36.0
